# SQL Server 2019 big data cluster Tutorial

## 01 - SQL Server Master Instance Queries

In this tutorial you will learn how to run standard SQL Server Queries against the Master Instance (MI) in a SQL Server big data cluster.

You'll start with a simple set of queries to explore the Instance:

In [ ]:
/* Instance Version */
SELECT @@VERSION; 
GO

/* General Configuration */
USE master;  
GO  
EXEC sp_configure;
GO

/* Databases on this Instance */
SELECT db.name AS 'Database Name'
, Physical_Name AS 'Location on Disk'
, Cast(Cast(Round(cast(mf.size as decimal) * 8.0/1024000.0,2) as decimal(18,2)) as nvarchar) 'Size (GB)'
FROM sys.master_files mf
INNER JOIN 
    sys.databases db ON db.database_id = mf.database_id
WHERE mf.type_desc = 'ROWS';
GO

SELECT * from sys.master_files


## 02 - Query Data

The SQL Server 2019 big data cluster Master Instance is a SQL Server Instance - and as such has most all of the query facilities and capabilities of Microsoft SQL Server running on Linux. 

Show The Populations from WorldWideImporters DB

In [ ]:

USE [WideWorldImporters-Full];
GO
/* Show the Populations. 
Where do we have the most people?
 */
SELECT TOP 10 CityName as 'City Name'
, StateProvinceName as 'State or Province'
, sp.LatestRecordedPopulation as 'Population'
, CountryName
FROM Application.Cities AS city
JOIN Application.StateProvinces AS sp ON
    city.StateProvinceID = sp.StateProvinceID
JOIN Application.Countries AS ctry ON 
    sp.CountryID=ctry.CountryID
ORDER BY Population, CityName;
GO

## 03- Query Data

Show Customer Sales in WorldWideImporters DB

In [ ]:
/* Show Customer Sales in WWI OLTP */
USE [WideWorldImporters-Full];
GO

SELECT TOP 10 s.CustomerID
, s.CustomerName
, sc.CustomerCategoryName
,  pp.FullName AS PrimaryContact
,  ap.FullName AS AlternateContact
,  s.PhoneNumber
,  s.FaxNumber
,  bg.BuyingGroupName
,  s.WebsiteURL
,  dm.DeliveryMethodName AS DeliveryMethod
,  c.CityName AS CityName
,  s.DeliveryLocation AS DeliveryLocation
,  s.DeliveryRun
,  s.RunPosition
FROM Sales.Customers AS s
    LEFT OUTER JOIN Sales.CustomerCategories AS sc
    ON s.CustomerCategoryID = sc.CustomerCategoryID
    LEFT OUTER JOIN [Application].People AS pp
    ON s.PrimaryContactPersonID = pp.PersonID
    LEFT OUTER JOIN [Application].People AS ap
    ON s.AlternateContactPersonID = ap.PersonID
    LEFT OUTER JOIN Sales.BuyingGroups AS bg
    ON s.BuyingGroupID = bg.BuyingGroupID
    LEFT OUTER JOIN [Application].DeliveryMethods AS dm
    ON s.DeliveryMethodID = dm.DeliveryMethodID
    LEFT OUTER JOIN [Application].Cities AS c
    ON s.DeliveryCityID = c.CityID
ORDER BY c.CityName

## 04- Query Data

Show Backup History

In [ ]:
USE master; 
GO
SELECT DB.name AS [database],
    CASE
        WHEN BK.[type] = 'D' THEN 'FULL'
        WHEN BK.[type] = 'I' THEN 'DIFF'
        WHEN BK.[type] = 'L' THEN 'LOG'
        WHEN BK.[type] = 'F' THEN 'FILEGROUP'
        WHEN BK.[type] = 'G' THEN 'DIFFERENTIAL - FILE'
        WHEN BK.[type] = 'P' THEN 'PARTIAL'
        WHEN BK.[type] = 'Q' THEN 'DIFFERENTIAL PARTIAL'
    END AS [backup_type],
    MAX(BK.backup_start_date) AS [last_backup]
FROM master..sysdatabases AS DB
LEFT JOIN msdb..backupset AS BK ON DB.name = BK.database_name
LEFT JOIN msdb..backupmediafamily AS MD ON BK.media_set_id = MD.media_set_id
GROUP BY DB.name, BK.[type]
ORDER BY MAX(BK.backup_start_date) DESC, DB.name;

## 05- Query Data

Show Query Performance

In [ ]:
USE master; 
go 
SELECT TOP 10
   rank() over(order by total_worker_time desc,sql_handle,statement_start_offset) as [rank]
   , creation_time as [create_time]
   , last_execution_time as [last_execution_time]
   , (total_worker_time+0.0)/1000 as [total_worker_time]
   , (total_worker_time+0.0)/(execution_count*1000) as [average_cpu_time]
   , total_logical_reads as [logical_reads]
   , total_logical_writes as [logical_writes]
   , execution_count as [execution_count]
   , total_logical_reads+total_logical_writes as [aggregated_io]
   , (total_logical_reads+total_logical_writes)/(execution_count + 0.0) as [average_io]
   , case when sql_handle IS NULL
       then ''
       else ( substring(st.text,(qs.statement_start_offset+2)/2,
           (case when qs.statement_end_offset = -1
               then len(convert(nvarchar(MAX),st.text))*2
               else qs.statement_end_offset
           end - qs.statement_start_offset) /2  ) )
   end as [query_text]
   , db_name(st.dbid) as [database]
   , st.objectid as [object_id]
from sys.dm_exec_query_stats qs
cross apply sys.dm_exec_sql_text(sql_handle) st
where total_worker_time > 0
order by total_worker_time  desc

## 06- Query Data

Show Server Memory by DB

In [ ]:


DECLARE @total_buffer INT;

SELECT @total_buffer = cntr_value
FROM sys.dm_os_performance_counters 
WHERE RTRIM([object_name]) LIKE '%Buffer Manager'
AND counter_name = 'Database Pages';

;WITH src AS
(
SELECT 
database_id, db_buffer_pages = COUNT_BIG(*)
FROM sys.dm_os_buffer_descriptors
--WHERE database_id BETWEEN 5 AND 32766
GROUP BY database_id
)
SELECT
[db_name] = CASE [database_id] WHEN 32767 
THEN 'Resource DB' 
ELSE DB_NAME([database_id]) END,
db_buffer_pages,
db_buffer_MB = db_buffer_pages / 128,
db_buffer_percent = CONVERT(DECIMAL(6,3), 
db_buffer_pages * 100.0 / @total_buffer)
FROM src
ORDER BY db_buffer_MB DESC; 

## 07- Query Data

Show Table Space  DB

In [ ]:
USE [WideWorldImporters-Full]; 
GO 

SELECT TOP(10)
SCH.name as schema_name, 
concat(TABL.name,':', INDX.name) AS table_name, 
--INDX.name AS index_name,
SUM(PART.rows) AS row_count,
(SUM(ALOC.used_pages)*8/1024) AS used_space_MB, 
(SUM(ALOC.total_pages)*8/1024) AS total_space_MB, 
(SUM(ALOC.data_pages)*8/1024) AS data_space_MB,
SUM(ALOC.total_pages) AS total_pages, 
SUM(ALOC.used_pages) AS used_pages, 
SUM(ALOC.data_pages) AS data_pages
FROM sys.Tables AS TABL 
INNER JOIN sys.Indexes AS INDX 
ON TABL.object_id = INDX.object_id 
INNER JOIN sys.Partitions AS PART 
ON INDX.object_id = PART.object_id AND INDX.index_id = PART.index_id 
INNER JOIN sys.Allocation_Units AS ALOC ON PART.partition_id = ALOC.container_id 
INNER JOIN sys.schemas as SCH on tabl.schema_id = sch.schema_id
GROUP BY SCH.name, TABL.name,
			INDX.object_id,
			INDX.index_id,
			INDX.name
ORDER BY (SUM(ALOC.total_pages)*8/1024) desc